# Growthcurves Tutorial

This tutorial is organized into two sections:

1. **Mechanistic** model fitting (ODE-based, parametric)
2. **Phenomenological** model fitting
   - **Parametric** phenomenological models (closed-form ln-space models)
   - **Non-parametric** phenomenological models (spline and sliding-window)

We'll fit each group, extract growth statistics, and visualize fitted curves.

In [ ]:
from pprint import pformat, pprint

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import growthcurves as gc

# Generate synthetic bacterial growth data with realistic noise
# This simulates OD600 measurements with lag, exponential, and stationary phases

# Set random seed for reproducibility
np.random.seed(42)

# Parameters for synthetic growth curve
n_points = 440  # Number of measurements
measurement_interval_minutes = 12  # Measurements every 12 minutes
time = np.array([(measurement_interval_minutes * n) / 60 for n in range(n_points)])

# Growth curve parameters (logistic model)
N0 = 0.05  # Initial OD
K = 0.50  # Carrying capacity (max OD)
mu = 0.15  # Growth rate (1/hour)
lag = 30.0  # Lag phase duration (hours)

# Generate base logistic growth curve
def logistic_growth(t, N0, K, mu, lag):
    """Generate logistic growth curve with lag phase"""
    # Adjust time for lag phase
    t_adj = t - lag
    # For times before lag ends, growth is minimal
    t_adj = np.maximum(t_adj, 0)
    # Logistic equation
    N = K / (1 + (K / N0 - 1) * np.exp(-mu * t_adj))
    return N

# Generate the clean curve
clean_curve = logistic_growth(time, N0, K, mu, lag)

# Add realistic noise that varies with growth phase
# Lag phase: very low noise
# Exponential phase: moderate noise, proportional to OD
# Stationary phase: slightly higher noise due to cell death/lysis
noise_std_base = 0.003  # Base noise level
relative_noise = 0.01  # Noise proportional to OD

# Generate heteroscedastic noise (noise increases with OD)
noise = np.random.normal(0, noise_std_base, n_points) + \
        np.random.normal(0, relative_noise * clean_curve, n_points)

# Add occasional "outliers" (simulate experimental artifacts)
outlier_probability = 0.02
outliers = np.random.random(n_points) < outlier_probability
noise[outliers] += np.random.normal(0, 0.01, outliers.sum())

# Generate final noisy data
data = clean_curve + noise

# Ensure data is non-negative (OD can't be negative)
data = np.maximum(data, 0.01)

# Convert to list for compatibility with rest of tutorial
data = data.tolist()

print(f"Generated {len(data)} synthetic OD600 measurements")
print(f"Time range: {time[0]:.1f} to {time[-1]:.1f} hours")
print(f"OD range: {min(data):.4f} to {max(data):.4f}")

## How Growth Parameters Are Calculated

The table below summarizes how the main reported growth statistics are calculated across model classes.

| Output key | Meaning | How it is calculated |
|---|---|---|
| `max_od` | Maximum observed/fitted OD | Maximum OD over the valid data range |
| `mu_max` | Maximum specific growth rate (μ_max) | Maximum of `d(ln N)/dt` from the fitted model (or local fit for non-parametric) |
| `intrinsic_growth_rate` | Intrinsic model rate parameter | For mechanistic models: fitted intrinsic `μ`; for phenomenological/non-parametric: `None` |
| `doubling_time` | Doubling time in hours | `ln(2) / mu_max` |
| `time_at_umax` | Time at maximum specific growth | Time where `mu_max` reaches its maximum |
| `od_at_umax` | OD at time of μ_max | Model-predicted OD at `time_at_umax` |
| `exp_phase_start`, `exp_phase_end` | Exponential phase boundaries | From threshold or tangent phase-boundary method in `extract_stats()` |
| `model_rmse` | Fit error | RMSE between observed OD and model-predicted OD over the model fit window |

For this tutorial:
- Mechanistic comparisons use mechanistic parametric fits.
- Phenomenological comparisons include both phenomenological parametric and non-parametric fits.


# Extract growth stats from the dataset

The `extract_stats_from_fit()` function calculates these key metrics:

- `max_od`: Maximum OD value within the fitted window
- `mu_max`: **Observed** maximum specific growth rate μ_max (hour⁻¹) - calculated from the fitted curve
- `intrinsic_growth_rate`: **Model parameter** for intrinsic growth rate (parametric models only, `None` for non-parametric)
- `doubling_time`: Time to double the population at peak growth (hours)
- `exp_phase_start`: When exponential phase begins (hours)
- `exp_phase_end`: When exponential phase ends (hours)
- `time_at_umax`: Time when μ reaches its maximum (hours)
- `od_at_umax`: OD value at time of maximum μ
- `fit_t_min`: Start of fitting window (hours)
- `fit_t_max`: End of fitting window (hours)
- `fit_method`: Identifier for the method used
- `model_rmse`: Root mean squared error

Descriptive parameters are extracted from the fits. Where parameters are not extracted directly from the fitted model, they are calculated. The table below shows how different stats are calculated according to the different approaches:

## MECHANISTIC MODELS

| Name | Model | Equation | Exp Start | Exp End | Intrinsic μ | μ max | Carrying Capacity | Fit |
|------|-------|----------|-----------|---------|-------------|-------|-------------------|-----|
| Logistic | parametric | `dN/dt = μ * (1 - N / K) * N` | threshold/<br>tangent | threshold/<br>tangent | μ | max dln(N)/dt | K | entire curve |
| Gompertz | parametric | `dN/dt = μ * math.log(K / N) * N` | threshold/<br>tangent | threshold/<br>tangent | μ | max dln(N)/dt | K | entire curve |
| Richards | parametric | `dN/dt = μ * (1 - (N / K)**beta) * N` | threshold/<br>tangent | threshold/<br>tangent | μ | max dln(N)/dt | A | entire curve |
| Baranyi | parametric | `dN/dt= μ * math.exp(μ * t) / (math.exp(h0) - 1 + math.exp(μ * t)) * (1 - N / K) * N` | threshold/<br>tangent | threshold/<br>tangent | μ | max dln(N)/dt | K | entire curve |

## PHENOMENOLOGICAL MODELS

| Name | Model | Equation | Exp Start | Exp End | Intrinsic μ | μ max | Max OD | Fit |
|------|-------|----------|-----------|---------|-------------|-------|--------|-----|
| Linear | non-parametric | `ln(N(t)) = N0 + b * t` | threshold/<br>tangent | threshold/<br>tangent | n.a. | b | max OD raw | only window |
| Spline | non-parametric | `ln(N(t)) = spline(t)` | threshold/<br>tangent | threshold/<br>tangent | n.a. | max of derivative of spline | max OD raw | only log phase |
| Logistic (phenom) | parametric | `ln(N(t)/N0) = A / (1 + exp(4 * μ_max * (λ - t) / A + 2))` | λ | threshold/<br>tangent | n.a. | μ_max | K | entire curve |
| Gompertz (phenom) | parametric | `ln(N(t)/N0) = A * exp(-exp(μ_max * exp(1) * (λ - t) / A + 1))` | λ | threshold/<br>tangent | n.a. | μ_max | K | entire curve |
| Gompertz (modified) | parametric | `ln(N(t)/N0) = A * exp(-exp(μ_max * exp(1) * (λ - t) / A + 1)) + A * exp(α * (t - t_shift))` | λ | threshold/<br>tangent | n.a. | μ_max | K | entire curve |
| Richards (phenom) | parametric | `ln(N(t)/N0) = A * (1 + ν * exp(1 + ν + μ_max * (1 + ν)**(1/ν) * (λ - t) / A))**(-1/ν)` | λ | threshold/<br>tangent | n.a. | μ_max | K | entire curve |

### Understanding Growth Rates: Intrinsic vs. Observed

**Important distinction:**

- **`mu_max`** (μ_max): The **observed** maximum specific growth rate calculated from the fitted curve as max(d(ln N)/dt). This is what you measure from the data.

- **`intrinsic_growth_rate`**: The **model parameter** representing intrinsic growth capacity:
  - **Parametric models**: This is a fitted parameter (e.g., `r` in Logistic, `mu_max` in Gompertz)
  - **Non-parametric methods**: Returns `None` (no model parameter exists)

In [57]:
def plot_growth_stats_comparison(stats_dict, title, metric_order=None):
    df = pd.DataFrame(stats_dict).T

    default_metrics = [
        "mu_max",
        "intrinsic_growth_rate",
        "doubling_time",
        "time_at_umax",
        "exp_phase_start",
        "exp_phase_end",
        "model_rmse",
    ]

    metrics = metric_order or [m for m in default_metrics if m in df.columns]
    numeric_df = df.copy()
    for m in metrics:
        numeric_df[m] = pd.to_numeric(numeric_df[m], errors="coerce")

    n_metrics = len(metrics)
    n_cols = 3
    n_rows = int(np.ceil(n_metrics / n_cols))

    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        subplot_titles=[m.replace("_", " ").title() for m in metrics],
        horizontal_spacing=0.08,
        vertical_spacing=0.15,
    )

    method_names = list(numeric_df.index)
    for i, metric in enumerate(metrics):
        row = i // n_cols + 1
        col = i % n_cols + 1
        fig.add_trace(
            go.Bar(
                x=method_names,
                y=numeric_df[metric].tolist(),
                showlegend=False,
                marker=dict(line=dict(color="black", width=1)),
            ),
            row=row,
            col=col,
        )

    fig.update_layout(
        title=title,
        height=max(420, 320 * n_rows),
        width=1200,
        bargap=0.25,
        template="plotly_white",
    )
    return fig

# Mechanistic Models

Mechanistic models are ODE-based parametric models that encode growth dynamics as differential equations. These models are used to model underlying growth processes and can be used to determine the intrinsic growth rate (μ) and carrying capacity (K) of a strain.

# Fit models to the data

Each fit returns a dictionary containing:
- `model_type`: The fitting method used
- `params`: Fitted model parameters (dictionary)

In [58]:
# Generate each fit independently
fit_mech_logistic = gc.parametric.fit_parametric(time, data, method="mech_logistic")
fit_mech_gompertz = gc.parametric.fit_parametric(time, data, method="mech_gompertz")
fit_mech_richards = gc.parametric.fit_parametric(time, data, method="mech_richards")
fit_mech_baranyi = gc.parametric.fit_parametric(time, data, method="mech_baranyi")

# Combine fits into a dictionary for easier iteration
mechanistic_fits = {
    "mech_logistic": fit_mech_logistic,
    "mech_gompertz": fit_mech_gompertz,
    "mech_richards": fit_mech_richards,
    "mech_baranyi": fit_mech_baranyi,
}

# Access and display a single fit result
print("=== Logistic Fit Result ===")

pprint(fit_mech_logistic, indent=2)

print("\n=== Accessing specific values ===")
print(f"Model type: {fit_mech_logistic['model_type']}")
print(f"Parameters:\n{pformat(fit_mech_logistic['params'], indent=2)}")

print("\n=== Accessing individual parameters ===")
params = fit_mech_logistic["params"]
print(f"Growth rate (μ): {params['mu']:.6f}")
print(f"Carrying capacity (K): {params['K']:.6f}")
print(f"Initial OD (N0): {params['N0']:.6f}")

=== Logistic Fit Result ===
{ 'model_type': 'mech_logistic',
  'params': { 'K': np.float64(0.4199540943446524),
              'N0': np.float64(1.8464709479959055e-06),
              'fit_t_max': 87.4,
              'fit_t_min': 0.0,
              'mu': np.float64(0.2634572595992891),
              'y0': np.float64(0.05196698646330492)}}

=== Accessing specific values ===
Model type: mech_logistic
Parameters:
{ 'K': np.float64(0.4199540943446524),
  'N0': np.float64(1.8464709479959055e-06),
  'fit_t_max': 87.4,
  'fit_t_min': 0.0,
  'mu': np.float64(0.2634572595992891),
  'y0': np.float64(0.05196698646330492)}

=== Accessing individual parameters ===
Growth rate (μ): 0.263457
Carrying capacity (K): 0.419954
Initial OD (N0): 0.000002


# Extract growth stats from the fit object

In [59]:
# Extract stats from each fit independently
stats_mech_logistic = gc.utils.extract_stats(fit_mech_logistic, time, data)
stats_mech_gompertz = gc.utils.extract_stats(fit_mech_gompertz, time, data)
stats_mech_richards = gc.utils.extract_stats(fit_mech_richards, time, data)
stats_mech_baranyi = gc.utils.extract_stats(fit_mech_baranyi, time, data)

# Combine stats into a dictionary
mechanistic_stats = {
    "mech_logistic": stats_mech_logistic,
    "mech_gompertz": stats_mech_gompertz,
    "mech_richards": stats_mech_richards,
    "mech_baranyi": stats_mech_baranyi,
}

# Display growth statistics for logistic fit
print("=== Logistic Growth Statistics ===")

pprint(stats_mech_logistic, indent=2)

# compare stats in a dataframe
mechanistic_df = pd.DataFrame(mechanistic_stats).T[
    [
        "mu_max",
        "intrinsic_growth_rate",
        "doubling_time",
        "time_at_umax",
        "exp_phase_start",
        "exp_phase_end",
        "model_rmse",
    ]
]
mechanistic_df.T

=== Logistic Growth Statistics ===
{ 'N0': 0.05196698646330492,
  'doubling_time': 5.207572043167674,
  'exp_phase_end': 56.00576093117492,
  'exp_phase_start': 29.711002069142133,
  'fit_method': 'model_fitting_mech_logistic',
  'fit_t_max': 87.4,
  'fit_t_min': 0.0,
  'intrinsic_growth_rate': 0.2634572595992891,
  'max_od': 0.47192108080795736,
  'model_rmse': 0.005409160816318739,
  'mu_max': 0.13310371413283725,
  'od_at_umax': 0.15769435217401157,
  'time_at_umax': 42.9118236472946}


,mech_logistic,mech_gompertz,mech_richards,mech_baranyi
mu_max,0.133104,0.082055,0.125975,0.132389
intrinsic_growth_rate,0.263457,0.049584,0.217648,0.263518
doubling_time,5.207572,8.447369,5.502276,5.235674
time_at_umax,42.911824,24.345892,43.437275,42.911824
exp_phase_start,29.711002,6.773672,28.098186,29.665206
exp_phase_end,56.005761,71.19749,55.895645,56.05596
model_rmse,0.005409,0.045415,0.004954,0.005408


# Visualize the fitted mechanistic models over the data

Each mechanistic model is plotted individually with:
- Raw data points (gray markers)
- Fitted model curve (blue line)
- Exponential phase shading (light green)
- Growth statistics annotations (N0 line, max OD line, μmax point, tangent line)

In [60]:
# Create individual plots for each mechanistic model

for model_name, fit in mechanistic_fits.items():
    stats = mechanistic_stats[model_name]

    # Create base plot with data
    fig = gc.plot.create_base_plot(time, data, scale="linear")

    # Annotate with fit and growth statistics
    fig = gc.plot.annotate_plot(
        fig,
        scale="linear",
        phase_boundaries=(stats["exp_phase_start"], stats["exp_phase_end"]),
        time_umax=stats["time_at_umax"],
        od_umax=stats["od_at_umax"],
        od_max=stats["max_od"],
        N0=stats["N0"],
        umax=stats["mu_max"],
        umax_point=(stats["time_at_umax"], stats["od_at_umax"]),
        fitted_model=fit,
        draw_umax_tangent=True,
    )

    # Add title and display
    model_titles = {
        "mech_logistic": "Mechanistic Logistic Model",
        "mech_gompertz": "Mechanistic Gompertz Model",
        "mech_richards": "Mechanistic Richards Model",
        "mech_baranyi": "Mechanistic Baranyi Model",
    }
    fig.update_layout(
        title=model_titles.get(model_name, model_name),
        height=500,
        width=800,
        template="plotly_white",
    )
    fig.show()

# Mechanistic Growth Statistics Comparison (Plotly)

This chart compares key growth statistics across mechanistic models.

In [61]:
fig_mech_stats = plot_growth_stats_comparison(
    mechanistic_stats,
    title="Mechanistic models: growth statistics comparison",
)
fig_mech_stats.show()

# Phenomenological Models

Phenomenological approaches focus on fitting observed growth behavior directly, without explicitly modeling underlying mechanisms. These models can be used to calculate descriptive statistics such as the maximum specific growth rate (maximum per capita growth rate, μ_max) but not the intrinsic growth rate.

# Parametric Models

These are phenomenological **parametric** models fit in ln-space.

# Fit models

In [62]:
# Generate each fit independently
fit_phenom_logistic = gc.parametric.fit_parametric(time, data, method="phenom_logistic")
fit_phenom_gompertz = gc.parametric.fit_parametric(time, data, method="phenom_gompertz")
fit_phenom_gompertz_modified = gc.parametric.fit_parametric(
    time, data, method="phenom_gompertz_modified"
)
fit_phenom_richards = gc.parametric.fit_parametric(time, data, method="phenom_richards")

# Combine fits into a dictionary
phenom_param_fits = {
    "phenom_logistic": fit_phenom_logistic,
    "phenom_gompertz": fit_phenom_gompertz,
    "phenom_gompertz_modified": fit_phenom_gompertz_modified,
    "phenom_richards": fit_phenom_richards,
}

# Example: Compare phenomenological logistic fit and stats
print("=== Phenomenological Logistic Fit ===")
pprint(fit_phenom_logistic, indent=2)

=== Phenomenological Logistic Fit ===
{ 'model_type': 'phenom_logistic',
  'params': { 'A': np.float64(2.2053922567445503),
              'N0': np.float64(0.052075083822616296),
              'fit_t_max': 87.4,
              'fit_t_min': 0.0,
              'lam': np.float64(34.77980413414468),
              'mu_max': np.float64(0.1357663399984608)}}


# Extract stats from fitted models

In [63]:
# Extract stats from each fit independently
stats_phenom_logistic = gc.utils.extract_stats(
    fit_phenom_logistic, time, data, phase_boundary_method="tangent"
)
stats_phenom_gompertz = gc.utils.extract_stats(
    fit_phenom_gompertz, time, data, phase_boundary_method="tangent"
)
stats_phenom_gompertz_modified = gc.utils.extract_stats(
    fit_phenom_gompertz_modified, time, data, phase_boundary_method="tangent"
)
stats_phenom_richards = gc.utils.extract_stats(
    fit_phenom_richards, time, data, phase_boundary_method="tangent"
)

# Combine stats into a dictionary
phenom_param_stats = {
    "phenom_logistic": stats_phenom_logistic,
    "phenom_gompertz": stats_phenom_gompertz,
    "phenom_gompertz_modified": stats_phenom_gompertz_modified,
    "phenom_richards": stats_phenom_richards,
}

phenom_param_df = pd.DataFrame(phenom_param_stats).T[
    [
        "mu_max",
        "intrinsic_growth_rate",
        "doubling_time",
        "time_at_umax",
        "exp_phase_start",
        "exp_phase_end",
        "model_rmse",
    ]
]

print("\n=== Phenomenological Logistic Stats ===")
pprint(stats_phenom_logistic, indent=2)

phenom_param_df


=== Phenomenological Logistic Stats ===
{ 'N0': 0.052075083822616296,
  'doubling_time': 5.105442045265444,
  'exp_phase_end': 51.02354925004566,
  'exp_phase_start': 34.77980413414468,
  'fit_method': 'model_fitting_phenom_logistic',
  'fit_t_max': 87.4,
  'fit_t_min': 0.0,
  'intrinsic_growth_rate': None,
  'max_od': 0.4725012622525146,
  'model_rmse': 0.005434496057147315,
  'mu_max': 0.1357663399984608,
  'od_at_umax': 0.1570777840090588,
  'time_at_umax': 42.9118236472946}


,mu_max,intrinsic_growth_rate,doubling_time,time_at_umax,exp_phase_start,exp_phase_end,model_rmse
phenom_logistic,0.135766,None,5.105442,42.911824,34.779804,51.023549,0.005434
phenom_gompertz,0.160909,None,4.307694,41.160321,36.223882,49.641522,0.007716
phenom_gompertz_modified,0.164217,None,4.220921,41.335471,36.564612,49.620311,0.007574
phenom_richards,0.12963,None,5.347136,43.612425,34.165938,51.433007,0.005156


# Visualize fits and calculated growth descriptors

Each phenomenological parametric model is plotted individually with:
- Raw data points (gray markers)
- Fitted model curve (blue line)
- Exponential phase shading (light green)
- Growth statistics annotations (N0 line, max OD line, μmax point, tangent line)

In [64]:
# Create individual plots for each phenomenological parametric model

for model_name, fit in phenom_param_fits.items():
    stats = phenom_param_stats[model_name]

    # Create base plot with data
    fig = gc.plot.create_base_plot(time, data, scale="log")

    # Annotate with fit and growth statistics
    fig = gc.plot.annotate_plot(
        fig,
        scale="log",
        phase_boundaries=(stats["exp_phase_start"], stats["exp_phase_end"]),
        time_umax=stats["time_at_umax"],
        od_umax=stats["od_at_umax"],
        od_max=stats["max_od"],
        N0=stats["N0"],
        umax=stats["mu_max"],
        umax_point=(stats["time_at_umax"], stats["od_at_umax"]),
        fitted_model=fit,
        draw_umax_tangent=True,
    )

    # Add title and display
    model_titles = {
        "phenom_logistic": "Phenomenological Logistic Model",
        "phenom_gompertz": "Phenomenological Gompertz Model",
        "phenom_gompertz_modified": "Phenomenological Gompertz (Modified) Model",
        "phenom_richards": "Phenomenological Richards Model",
    }
    fig.update_layout(
        title=model_titles.get(model_name, model_name),
        height=500,
        width=800,
        template="plotly_white",
    )
    fig.show()

# Non-Parametric Methods

These are phenomenological **non-parametric** fits that estimate growth features directly from local trends and smoothing. Spline and sliding window options are available. These methods only fit small sections of the data (sliding_window: linear model of Ln(N) across a user defined window of points; spline: spline curve fitted to the region around the exponential phase).

In [65]:
# Generate each fit independently
fit_spline = gc.non_parametric.fit_non_parametric(
    time,
    data,
    method="spline",
    spline_s=0.2,
)

fit_sliding_window = gc.non_parametric.fit_non_parametric(
    time,
    data,
    method="sliding_window",
    window_points=7,
)

# Combine fits into a dictionary
phenom_nonparam_fits = {
    "spline": fit_spline,
    "sliding_window": fit_sliding_window,
}

# Display non-parametric fit and stats results
pprint(phenom_nonparam_fits, indent=2)

{ 'sliding_window': { 'model_type': 'sliding_window',
                      'params': { 'fit_t_max': 45.4,
                                  'fit_t_min': 44.2,
                                  'intercept': -8.295980209369276,
                                  'slope': 0.14867963611003765,
                                  'time_at_umax': 44.8,
                                  'window_points': 7}},
  'spline': { 'model_type': 'spline',
              'params': { 'fit_t_max': 53.6,
                          'fit_t_min': 33.2,
                          'mu_max': 0.12313961981297095,
                          'spline_s': 0.2,
                          'tck_c': [ -2.7144906039298116,
                                     -2.5493994330828804,
                                     -1.0100998726918884,
                                     -0.9056673155505218,
                                     0.0,
                                     0.0,
                                     0.0,
           

# Extract stats from method object

In [66]:
# Extract stats from each fit independently
stats_spline = gc.utils.extract_stats(
    fit_spline,
    time,
    data,
    phase_boundary_method="tangent",
)

stats_sliding_window = gc.utils.extract_stats(
    fit_sliding_window,
    time,
    data,
    phase_boundary_method="tangent",
)

# Combine stats into a dictionary
phenom_nonparam_stats = {
    "spline": stats_spline,
    "sliding_window": stats_sliding_window,
}

phenom_nonparam_df = pd.DataFrame(phenom_nonparam_stats).T[
    [
        "mu_max",
        "intrinsic_growth_rate",
        "doubling_time",
        "time_at_umax",
        "exp_phase_start",
        "exp_phase_end",
        "model_rmse",
    ]
]
phenom_nonparam_df.T

,spline,sliding_window
mu_max,0.12314,0.14868
intrinsic_growth_rate,None,None
doubling_time,5.628953,4.662018
time_at_umax,43.143719,44.8
exp_phase_start,33.406067,35.499161
exp_phase_end,52.143152,51.017611
model_rmse,0.01979,0.001146


# Customizing phase boundary dectection

Two methods are available for determining exponential phase boundaries:

#### 1. **Threshold Method** (used by parametric models)
- Tracks the instantaneous specific growth rate μ(t)
- `exp_phase_start`: First time when μ exceeds a fraction of μ_max (default: 15%)
- `exp_phase_end`: First time after peak when μ drops below the threshold (default: 15%)

#### 2. **Tangent Method** (used by non-parametric methods)
- Constructs a tangent line **in log space** at the point of maximum growth rate
- Extends this tangent to intersect baseline (exp_phase_start) and plateau (exp_phase_end)
- Tangent line equation: `ln(OD(t)) = ln(OD_umax) + μ_max * (t - t_umax)`, or equivalently `OD(t) = OD_umax * exp(μ_max * (t - t_umax))`

In [67]:
# Compare phase-boundary methods and threshold sensitivity on the same fit
phase_boundary_rows = []

# Tangent method
stats_tangent = gc.utils.extract_stats(
    fit_spline,
    time,
    data,
    phase_boundary_method="tangent",
)
phase_boundary_rows.append(
    {
        "label": "tangent",
        "method": "tangent",
        "lag_frac": np.nan,
        "exp_frac": np.nan,
        "stats": stats_tangent,
    }
)

# Threshold method at low and high cutoffs
for frac, label in [(0.10, "threshold_low"), (0.30, "threshold_high")]:
    stats_threshold = gc.utils.extract_stats(
        fit_spline,
        time,
        data,
        phase_boundary_method="threshold",
        lag_frac=frac,
        exp_frac=frac,
    )
    phase_boundary_rows.append(
        {
            "label": label,
            "method": "threshold",
            "lag_frac": frac,
            "exp_frac": frac,
            "stats": stats_threshold,
        }
    )

phase_boundary_df = pd.DataFrame(
    [
        {
            "label": row["label"],
            "method": row["method"],
            "lag_frac": row["lag_frac"],
            "exp_frac": row["exp_frac"],
            "exp_phase_start": row["stats"]["exp_phase_start"],
            "exp_phase_end": row["stats"]["exp_phase_end"],
        }
        for row in phase_boundary_rows
    ]
)
phase_boundary_df

,label,method,lag_frac,exp_frac,exp_phase_start,exp_phase_end
0,tangent,tangent,NaN,NaN,33.406067,52.143152
1,threshold_low,threshold,0.1,0.1,27.508154,56.844539
2,threshold_high,threshold,0.3,0.3,33.080979,53.788160


In [68]:
def build_phase_plot(label, stats, fitted_model):
    fig = gc.plot.create_base_plot(time, data, scale="log")
    fig = gc.plot.annotate_plot(
        fig,
        scale="log",
        phase_boundaries=(stats["exp_phase_start"], stats["exp_phase_end"]),
        time_umax=stats["time_at_umax"],
        od_umax=stats["od_at_umax"],
        od_max=stats["max_od"],
        N0=stats["N0"],
        umax=stats["mu_max"],
        umax_point=(stats["time_at_umax"], stats["od_at_umax"]),
        fitted_model=fitted_model,
        draw_umax_tangent=True,
    )
    fig.update_layout(title=label)
    return fig


fig_tangent = build_phase_plot(
    "Spline fit + tangent phase boundaries", stats_tangent, fit_spline
)
fig_threshold_low = build_phase_plot(
    "Spline fit + threshold phase boundaries (low=0.10)",
    phase_boundary_rows[1]["stats"],
    fit_spline,
)
fig_threshold_high = build_phase_plot(
    "Spline fit + threshold phase boundaries (high=0.30)",
    phase_boundary_rows[2]["stats"],
    fit_spline,
)

fig_tangent.show()
fig_threshold_low.show()
fig_threshold_high.show()

### Quick Comparison Across Phenomenological Approaches

This table combines phenomenological parametric and non-parametric methods.

In [69]:
phenom_all_stats = {
    **phenom_param_stats,
    **phenom_nonparam_stats,
}

phenom_all_df = pd.DataFrame(phenom_all_stats).T[
    [
        "mu_max",
        "doubling_time",
        "time_at_umax",
        "exp_phase_start",
        "exp_phase_end",
        "model_rmse",
        "fit_method",
    ]
]

fig_phenom_stats = plot_growth_stats_comparison(
    phenom_all_stats,
    title="Phenomenological models: growth statistics comparison",
)

fig_phenom_stats.show()

phenom_all_df

,mu_max,doubling_time,time_at_umax,exp_phase_start,exp_phase_end,model_rmse,fit_method
phenom_logistic,0.135766,5.105442,42.911824,34.779804,51.023549,0.005434,model_fitting_phenom_logistic
phenom_gompertz,0.160909,4.307694,41.160321,36.223882,49.641522,0.007716,model_fitting_phenom_gompertz
phenom_gompertz_modified,0.164217,4.220921,41.335471,36.564612,49.620311,0.007574,model_fitting_phenom_gompertz_modified
phenom_richards,0.12963,5.347136,43.612425,34.165938,51.433007,0.005156,model_fitting_phenom_richards
spline,0.12314,5.628953,43.143719,33.406067,52.143152,0.01979,model_fitting_spline
sliding_window,0.14868,4.662018,44.8,35.499161,51.017611,0.001146,model_fitting_sliding_window


# Visualize the derivatives in linear (dOD/dt) and log (instantaneous μ) space

In [70]:
# Derivative metric plots for specific growth rate (mu) and dOD/dt
stats_for_derivative = gc.utils.extract_stats(
    fit_spline,
    time,
    data,
    phase_boundary_method="tangent",
)
phase_bounds = (
    stats_for_derivative["exp_phase_start"],
    stats_for_derivative["exp_phase_end"],
)

fig_mu = gc.plot.plot_derivative_metric(
    time,
    data,
    metric="mu",
    fit_result=fit_spline,
    phase_boundaries=phase_bounds,
    title="Specific growth rate (mu)",
)

fig_doddt = gc.plot.plot_derivative_metric(
    time,
    data,
    metric="dndt",
    fit_result=fit_spline,
    phase_boundaries=phase_bounds,
    title="First derivative (dOD/dt)",
)

fig_mu.show()
fig_doddt.show()